# Home Depot Product Search Relevance
The challenge is to predict a relevance score for the provided combinations of search terms and products. To create the ground truth labels, Home Depot has crowdsourced the search/product pairs to multiple human raters.

## LabGraph Create
This notebook uses the LabGraph create machine learning iPython module. You need a personal licence to run this code.

In [1]:
import graphlab as gl

### Load training, testing set and product descriptions

In [2]:
train = gl.SFrame.read_csv("../data/train.csv")

[INFO] This non-commercial license of GraphLab Create is assigned to thomasv1000@hotmail.fr and will expire on October 12, 2016. For commercial licensing options, visit https://dato.com/buy/.

[INFO] Start server at: ipc:///tmp/graphlab_server-25352 - Server binary: /Users/tjaskula/.graphlab/anaconda/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1454625506.log
[INFO] GraphLab Server Version: 1.8.1


PROGRESS: Finished parsing file /Users/tjaskula/Documents/GitHub/Kaggle.HomeDepot/data/train.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.103783 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,str,str,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/tjaskula/Documents/GitHub/Kaggle.HomeDepot/data/train.csv
PROGRESS: Parsing completed. Parsed 74067 lines in 0.156674 secs.


In [3]:
train.head()

id,product_uid,product_title,search_term,relevance
2,100001,Simpson Strong-Tie12-Gauge Angle ...,angle bracket,3.0
3,100001,Simpson Strong-Tie12-Gauge Angle ...,l bracket,2.5
9,100002,BEHR Premium TexturedDeckOver 1-gal. #SC-141 ...,deck over,3.0
16,100005,Delta Vero 1-HandleShower Only Faucet Trim ...,rain shower head,2.33
17,100005,Delta Vero 1-HandleShower Only Faucet Trim ...,shower only faucet,2.67
18,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,convection otr,3.0
20,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,microwave over stove,2.67
21,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,microwaves,3.0
23,100007,Lithonia Lighting Quantum2-Light Black LED ...,emergency light,2.67
27,100009,House of Fara 3/4 in. x 3in. x 8 ft. MDF Fluted ...,mdf 3/4,3.0


In [4]:
test = gl.SFrame.read_csv("../data/test.csv")

PROGRESS: Finished parsing file /Users/tjaskula/Documents/GitHub/Kaggle.HomeDepot/data/test.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.201795 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,int,str,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Finished parsing file /Users/tjaskula/Documents/GitHub/Kaggle.HomeDepot/data/test.csv
PROGRESS: Parsing completed. Parsed 166693 lines in 0.326212 secs.


In [5]:
test.head()

id,product_uid,product_title,search_term
1,100001,Simpson Strong-Tie12-Gauge Angle ...,90 degree bracket
4,100001,Simpson Strong-Tie12-Gauge Angle ...,metal l brackets
5,100001,Simpson Strong-Tie12-Gauge Angle ...,simpson sku able
6,100001,Simpson Strong-Tie12-Gauge Angle ...,simpson strong ties
7,100001,Simpson Strong-Tie12-Gauge Angle ...,simpson strong tie hcc668
8,100001,Simpson Strong-Tie12-Gauge Angle ...,wood connectors
10,100003,STERLING Ensemble 33-1/4in. x 60 in. x 75-1/4 ...,bath and shower kit
11,100003,STERLING Ensemble 33-1/4in. x 60 in. x 75-1/4 ...,bath drain kit
12,100003,STERLING Ensemble 33-1/4in. x 60 in. x 75-1/4 ...,one piece tub shower
13,100004,Grape Solar 265-WattPolycrystalline Solar ...,solar panel


In [6]:
desc = gl.SFrame.read_csv("../data/product_descriptions.csv")

PROGRESS: Finished parsing file /Users/tjaskula/Documents/GitHub/Kaggle.HomeDepot/data/product_descriptions.csv
PROGRESS: Parsing completed. Parsed 100 lines in 0.483037 secs.
------------------------------------------------------
Inferred types from first line of file as 
column_type_hints=[int,str]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------
PROGRESS: Read 61134 lines. Lines per second: 61139.3
PROGRESS: Finished parsing file /Users/tjaskula/Documents/GitHub/Kaggle.HomeDepot/data/product_descriptions.csv
PROGRESS: Parsing completed. Parsed 124428 lines in 1.58761 secs.


In [7]:
desc.head()

product_uid,product_description
100001,"Not only do angles makejoints stronger, they ..."
100002,BEHR Premium TexturedDECKOVER is an innova ...
100003,Classic architecturemeets contemporary de ...
100004,The Grape Solar 265-WattPolycrystalline PV Solar ...
100005,Update your bathroom withthe Delta Vero Single- ...
100006,Achieving deliciousresults is almost ...
100007,The Quantum Adjustable2-Light LED Black ...
100008,The Teks #10 x 1-1/2 in.Zinc-Plated Steel Was ...
100009,Get the House of Fara 3/4in. x 3 in. x 8 ft. MDF ...
100010,Valley View IndustriesMetal Stakes (4-Pack) ...


In [8]:
gl.canvas.set_target('ipynb')

In [9]:
train = train.join(desc, on = 'product_uid', how = 'left')

In [10]:
train.head()

id,product_uid,product_title,search_term,relevance,product_description
2,100001,Simpson Strong-Tie12-Gauge Angle ...,angle bracket,3.0,"Not only do angles makejoints stronger, they ..."
3,100001,Simpson Strong-Tie12-Gauge Angle ...,l bracket,2.5,"Not only do angles makejoints stronger, they ..."
9,100002,BEHR Premium TexturedDeckOver 1-gal. #SC-141 ...,deck over,3.0,BEHR Premium TexturedDECKOVER is an innova ...
16,100005,Delta Vero 1-HandleShower Only Faucet Trim ...,rain shower head,2.33,Update your bathroom withthe Delta Vero Single- ...
17,100005,Delta Vero 1-HandleShower Only Faucet Trim ...,shower only faucet,2.67,Update your bathroom withthe Delta Vero Single- ...
18,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,convection otr,3.0,Achieving deliciousresults is almost ...
20,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,microwave over stove,2.67,Achieving deliciousresults is almost ...
21,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,microwaves,3.0,Achieving deliciousresults is almost ...
23,100007,Lithonia Lighting Quantum2-Light Black LED ...,emergency light,2.67,The Quantum Adjustable2-Light LED Black ...
27,100009,House of Fara 3/4 in. x 3in. x 8 ft. MDF Fluted ...,mdf 3/4,3.0,Get the House of Fara 3/4in. x 3 in. x 8 ft. MDF ...


In [11]:
test = test.join(desc, on = 'product_uid', how = 'left')

In [12]:
test.head()

id,product_uid,product_title,search_term,product_description
1,100001,Simpson Strong-Tie12-Gauge Angle ...,90 degree bracket,"Not only do angles makejoints stronger, they ..."
4,100001,Simpson Strong-Tie12-Gauge Angle ...,metal l brackets,"Not only do angles makejoints stronger, they ..."
5,100001,Simpson Strong-Tie12-Gauge Angle ...,simpson sku able,"Not only do angles makejoints stronger, they ..."
6,100001,Simpson Strong-Tie12-Gauge Angle ...,simpson strong ties,"Not only do angles makejoints stronger, they ..."
7,100001,Simpson Strong-Tie12-Gauge Angle ...,simpson strong tie hcc668,"Not only do angles makejoints stronger, they ..."
8,100001,Simpson Strong-Tie12-Gauge Angle ...,wood connectors,"Not only do angles makejoints stronger, they ..."
10,100003,STERLING Ensemble 33-1/4in. x 60 in. x 75-1/4 ...,bath and shower kit,Classic architecturemeets contemporary de ...
11,100003,STERLING Ensemble 33-1/4in. x 60 in. x 75-1/4 ...,bath drain kit,Classic architecturemeets contemporary de ...
12,100003,STERLING Ensemble 33-1/4in. x 60 in. x 75-1/4 ...,one piece tub shower,Classic architecturemeets contemporary de ...
13,100004,Grape Solar 265-WattPolycrystalline Solar ...,solar panel,The Grape Solar 265-WattPolycrystalline PV Solar ...


In [13]:
train['search_term_word_count'] = gl.text_analytics.count_words(train['search_term'])
train_search_tfidf = gl.text_analytics.tf_idf(train['search_term_word_count'])

In [14]:
train['search_tfidf'] = train_search_tfidf

In [15]:
train.head()

id,product_uid,product_title,search_term,relevance,product_description
2,100001,Simpson Strong-Tie12-Gauge Angle ...,angle bracket,3.0,"Not only do angles makejoints stronger, they ..."
3,100001,Simpson Strong-Tie12-Gauge Angle ...,l bracket,2.5,"Not only do angles makejoints stronger, they ..."
9,100002,BEHR Premium TexturedDeckOver 1-gal. #SC-141 ...,deck over,3.0,BEHR Premium TexturedDECKOVER is an innova ...
16,100005,Delta Vero 1-HandleShower Only Faucet Trim ...,rain shower head,2.33,Update your bathroom withthe Delta Vero Single- ...
17,100005,Delta Vero 1-HandleShower Only Faucet Trim ...,shower only faucet,2.67,Update your bathroom withthe Delta Vero Single- ...
18,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,convection otr,3.0,Achieving deliciousresults is almost ...
20,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,microwave over stove,2.67,Achieving deliciousresults is almost ...
21,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,microwaves,3.0,Achieving deliciousresults is almost ...
23,100007,Lithonia Lighting Quantum2-Light Black LED ...,emergency light,2.67,The Quantum Adjustable2-Light LED Black ...
27,100009,House of Fara 3/4 in. x 3in. x 8 ft. MDF Fluted ...,mdf 3/4,3.0,Get the House of Fara 3/4in. x 3 in. x 8 ft. MDF ...


In [19]:
train['product_desc_word_count'] = gl.text_analytics.count_words(train['product_description'])
train_desc_tfidf = gl.text_analytics.tf_idf(train['product_desc_word_count'])

In [20]:
train['desc_tfidf'] = train_desc_tfidf

In [21]:
train.head()

id,product_uid,product_title,search_term,relevance,product_description
2,100001,Simpson Strong-Tie12-Gauge Angle ...,angle bracket,3.0,"Not only do angles makejoints stronger, they ..."
3,100001,Simpson Strong-Tie12-Gauge Angle ...,l bracket,2.5,"Not only do angles makejoints stronger, they ..."
9,100002,BEHR Premium TexturedDeckOver 1-gal. #SC-141 ...,deck over,3.0,BEHR Premium TexturedDECKOVER is an innova ...
16,100005,Delta Vero 1-HandleShower Only Faucet Trim ...,rain shower head,2.33,Update your bathroom withthe Delta Vero Single- ...
17,100005,Delta Vero 1-HandleShower Only Faucet Trim ...,shower only faucet,2.67,Update your bathroom withthe Delta Vero Single- ...
18,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,convection otr,3.0,Achieving deliciousresults is almost ...
20,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,microwave over stove,2.67,Achieving deliciousresults is almost ...
21,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,microwaves,3.0,Achieving deliciousresults is almost ...
23,100007,Lithonia Lighting Quantum2-Light Black LED ...,emergency light,2.67,The Quantum Adjustable2-Light LED Black ...
27,100009,House of Fara 3/4 in. x 3in. x 8 ft. MDF Fluted ...,mdf 3/4,3.0,Get the House of Fara 3/4in. x 3 in. x 8 ft. MDF ...


In [39]:
train['product_title_word_count'] = gl.text_analytics.count_words(train['product_title'])
train_title_tfidf = gl.text_analytics.tf_idf(train['product_title_word_count'])
train['title_tfidf'] = train_title_tfidf
train.head()

id,product_uid,product_title,search_term,relevance,product_description
2,100001,Simpson Strong-Tie12-Gauge Angle ...,angle bracket,3.0,"Not only do angles makejoints stronger, they ..."
3,100001,Simpson Strong-Tie12-Gauge Angle ...,l bracket,2.5,"Not only do angles makejoints stronger, they ..."
9,100002,BEHR Premium TexturedDeckOver 1-gal. #SC-141 ...,deck over,3.0,BEHR Premium TexturedDECKOVER is an innova ...
16,100005,Delta Vero 1-HandleShower Only Faucet Trim ...,rain shower head,2.33,Update your bathroom withthe Delta Vero Single- ...
17,100005,Delta Vero 1-HandleShower Only Faucet Trim ...,shower only faucet,2.67,Update your bathroom withthe Delta Vero Single- ...
18,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,convection otr,3.0,Achieving deliciousresults is almost ...
20,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,microwave over stove,2.67,Achieving deliciousresults is almost ...
21,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,microwaves,3.0,Achieving deliciousresults is almost ...
23,100007,Lithonia Lighting Quantum2-Light Black LED ...,emergency light,2.67,The Quantum Adjustable2-Light LED Black ...
27,100009,House of Fara 3/4 in. x 3in. x 8 ft. MDF Fluted ...,mdf 3/4,3.0,Get the House of Fara 3/4in. x 3 in. x 8 ft. MDF ...


In [40]:
train['distance'] = train.apply(lambda x: gl.distances.cosine(x['search_tfidf'],x['desc_tfidf']))
train['distance2'] = train.apply(lambda x: gl.distances.cosine(x['search_tfidf'],x['title_tfidf']))

In [41]:
train.head()

id,product_uid,product_title,search_term,relevance,product_description
2,100001,Simpson Strong-Tie12-Gauge Angle ...,angle bracket,3.0,"Not only do angles makejoints stronger, they ..."
3,100001,Simpson Strong-Tie12-Gauge Angle ...,l bracket,2.5,"Not only do angles makejoints stronger, they ..."
9,100002,BEHR Premium TexturedDeckOver 1-gal. #SC-141 ...,deck over,3.0,BEHR Premium TexturedDECKOVER is an innova ...
16,100005,Delta Vero 1-HandleShower Only Faucet Trim ...,rain shower head,2.33,Update your bathroom withthe Delta Vero Single- ...
17,100005,Delta Vero 1-HandleShower Only Faucet Trim ...,shower only faucet,2.67,Update your bathroom withthe Delta Vero Single- ...
18,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,convection otr,3.0,Achieving deliciousresults is almost ...
20,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,microwave over stove,2.67,Achieving deliciousresults is almost ...
21,100006,Whirlpool 1.9 cu. ft.Over the Range Convec ...,microwaves,3.0,Achieving deliciousresults is almost ...
23,100007,Lithonia Lighting Quantum2-Light Black LED ...,emergency light,2.67,The Quantum Adjustable2-Light LED Black ...
27,100009,House of Fara 3/4 in. x 3in. x 8 ft. MDF Fluted ...,mdf 3/4,3.0,Get the House of Fara 3/4in. x 3 in. x 8 ft. MDF ...


In [42]:
model1 = gl.linear_regression.create(train, target = 'relevance', features = ['distance', 'distance2'], validation_set = None)

PROGRESS: Linear regression:
PROGRESS: --------------------------------------------------------
PROGRESS: Number of examples          : 74067
PROGRESS: Number of features          : 2
PROGRESS: Number of unpacked features : 2
PROGRESS: Number of coefficients    : 3
PROGRESS: Starting Newton Method
PROGRESS: --------------------------------------------------------
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | Iteration | Passes   | Elapsed Time | Training-max_error | Training-rmse |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: | 1         | 2        | 0.056264     | 1.917518           | 0.510175      |
PROGRESS: +-----------+----------+--------------+--------------------+---------------+
PROGRESS: SUCCESS: Optimal solution found.
PROGRESS:


In [43]:
#let's take a look at the weights before we plot
model1.get("coefficients")

name,index,value,stderr
(intercept),None,3.32490615945,0.0148462020716
distance,None,-0.483754540522,0.019557894819
distance2,None,-0.680280391407,0.0122246518296


In [44]:
import matplotlib.pyplot as plt
%matplotlib inline

In [51]:
test['search_term_word_count'] = gl.text_analytics.count_words(test['search_term'])
test_search_tfidf = gl.text_analytics.tf_idf(test['search_term_word_count'])
test['search_tfidf'] = test_search_tfidf
test['product_desc_word_count'] = gl.text_analytics.count_words(test['product_description'])
test_desc_tfidf = gl.text_analytics.tf_idf(test['product_desc_word_count'])
test['desc_tfidf'] = test_desc_tfidf
test['product_title_word_count'] = gl.text_analytics.count_words(test['product_title'])
test_title_tfidf = gl.text_analytics.tf_idf(test['product_title_word_count'])
test['title_tfidf'] = test_title_tfidf
test['distance'] = test.apply(lambda x: gl.distances.cosine(x['search_tfidf'],x['desc_tfidf']))
test['distance2'] = test.apply(lambda x: gl.distances.cosine(x['search_tfidf'],x['title_tfidf']))

In [52]:
output = model1.predict(test)

In [54]:
output

dtype: float
Rows: 166693
[2.182411862496898, 2.1608712275203783, 2.3612125700089743, 2.365694400860675, 2.327752636506774, 2.1608712275203783, 2.397746858777891, 2.4062294292646813, 2.20256767865483, 2.6653424245644235, 2.495902336883988, 2.3947184513708013, 2.560457060750479, 2.370117441318758, 2.2524170869124873, 2.4402792166022387, 2.1608712275203783, 2.662017921606516, 2.1887854160501306, 2.2254033032766607, 2.783809966980468, 2.7734059187361826, 2.19674537252042, 2.3704196068919705, 2.1608712275203783, 2.2913664707704307, 2.1608712275203783, 2.3505437115707037, 2.2602475521528147, 2.356852855461554, 2.5091315095820987, 2.4004604855996465, 2.2254887509340744, 2.3663581592018677, 2.212200808279051, 2.1608712275203783, 2.2682759008387245, 2.186676091069286, 2.6675052500082996, 2.56677968678039, 2.1820546633534215, 2.3343837734655715, 2.228823856157994, 2.1608712275203783, 2.4937099839575474, 2.4098370450522046, 2.4645844275412947, 2.5113794031454444, 2.6280286079925754, 2.2905166478

In [88]:
submission = gl.SFrame(test['id'])

In [89]:
submission.add_column(output)
submission.rename({'X1': 'id', 'X2':'relevance'})

id,relevance
1,2.1824118625
4,2.16087122752
5,2.36121257001
6,2.36569440086
7,2.32775263651
8,2.16087122752
10,2.39774685878
11,2.40622942926
12,2.20256767865
13,2.66534242456


In [90]:
submission['relevance'] = submission.apply(lambda x: 3.0 if x['relevance'] > 3.0 else x['relevance'])
submission['relevance'] = submission.apply(lambda x: 1.0 if x['relevance'] < 1.0 else x['relevance'])

In [91]:
submission['relevance'] = submission.apply(lambda x: str(x['relevance']))

In [92]:
submission.export_csv('../data/submission.csv', quote_level = 3)